In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.model_selection

In [6]:
df = pd.read_csv("xAPI-Edu-Data.csv")
df.head()

,gender,NationalITy,PlaceofBirth,StageID,GradeID,SectionID,Topic,Semester,Relation,raisedhands,VisITedResources,AnnouncementsView,Discussion,ParentAnsweringSurvey,ParentschoolSatisfaction,StudentAbsenceDays,Class
0,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,15,16,2,20,Yes,Good,Under-7,M
1,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,20,20,3,25,Yes,Good,Under-7,M
2,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,10,7,0,30,No,Bad,Above-7,L
3,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,30,25,5,35,No,Bad,Above-7,L
4,M,KW,KuwaIT,lowerlevel,G-04,A,IT,F,Father,40,50,12,50,No,Bad,Above-7,M


## Basic Logistic Regression
1. Split the data into a training and test (hold-out) set
2. Train on the training set, and test for accuracy on the testing set

In [17]:
df['TotalQ'] = df['Class']
df['TotalQ'].loc[df.TotalQ == 'Low-Level'] = 0.0
df['TotalQ'].loc[df.TotalQ == 'Middle-Level'] = 1.0
df['TotalQ'].loc[df.TotalQ == 'High-Level'] = 2.0

continuous_subset = df.ix[:,9:13]
continuous_subset['gender'] = np.where(df['gender']=='M',1,0)
continuous_subset['Parent'] = np.where(df['Relation']=='Father',1,0)

X = np.array(continuous_subset).astype('float64')
y = np.array(df['TotalQ'])
X.shape

/Users/kpham/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


(480, 6)

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Split the data into a training and test set.
X_train, X_test, y_train, y_test = train_test_split(
         X, y)

lr = LogisticRegression()
# Fit the model on the trainng data.
lr.fit(X_train, y_train)
# Print the accuracy from the testing data.
print(accuracy_score(lr.predict(X_train), y_train))

0.683333333333


## Tuning the Model

Notes from Logistic Regression Project:

The model has some hyperparameters we can tune for hopefully better performance. For tuning the parameters of your model, you will use a mix of *cross-validation* and *grid search*. In Logistic Regression, the most important parameter to tune is the *regularization parameter* `C`. Note that the regularization parameter is not always part of the logistic regression model. 

The regularization parameter is used to control for unlikely high regression coefficients, and in other cases can be used when data is sparse, as a method of feature selection.

You will now implement some code to perform model tuning and selecting the regularization parameter $C$.

We use the following `cv_score` function to perform K-fold cross-validation and apply a scoring function to each test fold. In this incarnation we use accuracy score as the default scoring function.

In [38]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

def cv_score(lr, x, y, score_func=accuracy_score):
    result = 0
    nfold = 5
    for train, test in KFold(nfold).split(x): # split data into train/test groups, 5 times
        lr.fit(x[train], y[train]) # fit
        result += score_func(lr.predict(x[test]), y[test]) # evaluate score function on held-out data
    return result / nfold # average

In [39]:
lr = LogisticRegression()
score = cv_score(lr, X_train, y_train)
print(score) #accuracy score

0.652777777778


In [50]:
#the grid of parameters to search over
Cs = [0.001, 0.1, 1, 10, 100]

# your turn
best_score=0

for C in Cs:
    clf=LogisticRegression(C=C)
    score=cv_score(clf, X_test, y_test)
    print(C, score)

    if score > best_score:
        best_score = score
        best_C =C
print('best score, C:', best_score, best_C)

0.001 0.341666666667
0.1 0.625
1 0.675
10 0.675
100 0.675
best score, C: 0.675 1


In [53]:
# what defines Cs?

In [51]:
lr = LogisticRegression()
score = cv_score(lr, X_train, y_train)
print(accuracy_score(lr.predict(X_test), y_test)) #accuracy score of fitted model on test data

0.716666666667


## Useing scikit-learn's GridSearchCV tool to perform cross validation and grid search.

In [52]:
from sklearn.grid_search import GridSearchCV
param_grid = {'C' : Cs }
x = GridSearchCV(LogisticRegression(), param_grid, scoring = 'accuracy', cv = 5)
x.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.1, 1, 10, 100]}, pre_dispatch='2*n_jobs',
       refit=True, scoring='accuracy', verbose=0)

In [54]:
print(x.best_params_['C'])

1


In [55]:
print(accuracy_score(x.predict(X_test), y_test))

0.716666666667


## Math behind Logistic Regression

In [56]:
def cv_optimize(clf, parameters, Xtrain, ytrain, n_folds=5):
    gs = sklearn.model_selection.GridSearchCV(clf, param_grid=parameters, cv=n_folds)
    gs.fit(Xtrain, ytrain)
    print("BEST PARAMS", gs.best_params_)
    best = gs.best_estimator_
    return best

In [57]:
from sklearn.model_selection import train_test_split

def do_classify(clf, parameters, indf, featurenames, targetname, target1val, standardize=False, train_size=0.8):
    subdf=indf[featurenames]
    if standardize:
        subdfstd=(subdf - subdf.mean())/subdf.std()
    else:
        subdfstd=subdf
    X=subdfstd.values
    y=(indf[targetname].values==target1val)*1
    Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=train_size)
    clf = cv_optimize(clf, parameters, Xtrain, ytrain)
    clf=clf.fit(Xtrain, ytrain)
    training_accuracy = clf.score(Xtrain, ytrain)
    test_accuracy = clf.score(Xtest, ytest)
    print("Accuracy on training data: {:0.2f}".format(training_accuracy))
    print("Accuracy on test data:     {:0.2f}".format(test_accuracy))
    return clf, Xtrain, ytrain, Xtest, ytest

In [63]:
clf_l, Xtrain_l, ytrain_l, Xtest_l, ytest_l  = do_classify(LogisticRegression(), 
                                                           {"C": [0.01, 0.1, 1, 10, 100]}, 
                                                           # df, X, y, df.Class)

KeyError: '[(15.0, 16.0, 2.0, 20.0, 1.0, 1.0) (20.0, 20.0, 3.0, 25.0, 1.0, 1.0)\n (10.0, 7.0, 0.0, 30.0, 1.0, 1.0) (30.0, 25.0, 5.0, 35.0, 1.0, 1.0)\n (40.0, 50.0, 12.0, 50.0, 1.0, 1.0) (42.0, 30.0, 13.0, 70.0, 0.0, 1.0)\n (35.0, 12.0, 0.0, 17.0, 1.0, 1.0) (50.0, 10.0, 15.0, 22.0, 1.0, 1.0)\n (12.0, 21.0, 16.0, 50.0, 0.0, 1.0) (70.0, 80.0, 25.0, 70.0, 0.0, 1.0)\n (50.0, 88.0, 30.0, 80.0, 1.0, 1.0) (19.0, 6.0, 19.0, 12.0, 1.0, 1.0)\n (5.0, 1.0, 0.0, 11.0, 1.0, 1.0) (20.0, 14.0, 12.0, 19.0, 1.0, 1.0)\n (62.0, 70.0, 44.0, 60.0, 0.0, 0.0) (30.0, 40.0, 22.0, 66.0, 0.0, 1.0)\n (36.0, 30.0, 20.0, 80.0, 1.0, 1.0) (55.0, 13.0, 35.0, 90.0, 1.0, 1.0)\n (69.0, 15.0, 36.0, 96.0, 0.0, 0.0) (70.0, 50.0, 40.0, 99.0, 1.0, 0.0)\n (60.0, 60.0, 33.0, 90.0, 0.0, 1.0) (10.0, 12.0, 4.0, 80.0, 0.0, 1.0)\n (15.0, 21.0, 2.0, 90.0, 1.0, 1.0) (2.0, 0.0, 2.0, 50.0, 1.0, 1.0)\n (0.0, 2.0, 3.0, 70.0, 1.0, 1.0) (8.0, 7.0, 30.0, 40.0, 1.0, 1.0)\n (19.0, 19.0, 25.0, 40.0, 1.0, 1.0) (25.0, 15.0, 12.0, 33.0, 1.0, 1.0)\n 